In [1]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=7GgeC8plKn5IzNrfq2g9CyJ1wprQNU&access_type=offline&code_challenge=3g17ipZ8E7STNP2GEu5DMVISZMwSzYdx9v9I8XFzmho&code_challenge_method=S256


Credentials saved to file: [C:\Users\Vadim\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "sunny-lightning-392907" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the proje

In [2]:
!gcloud auth application-default set-quota-project sunny-lightning-392907


Credentials saved to file: [C:\Users\Vadim\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "sunny-lightning-392907" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [4]:
import speech_recognition as sr

In [5]:
r = sr.Recognizer()

In [6]:
context = """
You are an assistant who helps users understand more about our application. You need to answer in one or two sentences.
Users will ask you questions and you will answer them given the below information. Don't answer in more than three sentences:

About the application:
Our application is an outlook plugin which helps users to analyze and write their emails more efficiently. it has the following functions:
1. It can show a sentiment of email, i.e. negative or positive. This will help users to avoid sending negative emails and try to write more positive emails
2. It can provide a language tone of email, this should help users to avoid impolite words and avoid using inappropriate language
3. It can summarize long emails in a few sentences. This can save time for users and avoid reading long email chains
4. It can rewrite an email in more polite and positive language.
5. It can translate to other languages

User question:
"""

In [7]:
from vertexai.preview.language_models import TextGenerationModel
import vertexai

def send_user_input(project_id, location, question, temperature: float = .1):
    vertexai.init(project=project_id, location=location)
    
    parameters = {
        "temperature": temperature,
        "max_output_tokens": 64,
        "top_p": .8,
        "top_k": 40,
    }

    model = TextGenerationModel.from_pretrained("text-bison@001")
    request_text = context + question 
    #print(request_text)
    response = model.predict(request_text, **parameters)
    #print(f"Response from Model: {response.text}")
    return response.text
   


ModuleNotFoundError: No module named 'vertexai'

In [8]:
output_voice_file="response_new2.mp3"
output_voice_file_changed = "response_new2_changed.mp3"
idle_video_file = "idle9s.mp4"
#input_video_file = "listen.mp4"
listen_video_file = "listen_long.mp4"
speaking_video_file = "video_assistant.mp4"

In [11]:
from gtts import gTTS
from playsound import playsound
import cv2
import librosa
import soundfile as sf
import IPython
from IPython.display import Audio
from threading import Thread
from enum import Enum
from pygame import mixer
import os

class VoiceRecordingThread(Thread):
    def run(self):
        with sr.Microphone(device_index=0) as source:
            r.adjust_for_ambient_noise(source)
            #frame = printText(frame, "speak")
            #cv2.imshow('Unicorn', frame)
            #quitButton = cv2.waitKey(25) & 0xFF == ord('q')
            print("speak")
            audio = r.listen(source)
        user_input = r.recognize_google(audio)
        print("User input:", user_input)
        self.response = send_user_input('sunny-lightning-392907', 'us-central1', user_input)
        print("Model response:", self.response)
        voice_prep(self.response)
        
class PlaySoundThread(Thread):
    def run(self):
        save_shifted_pitch(output_voice_file, output_voice_file_changed, 7)
        #save_shifted_pitch("response", 7, )
        mixer.init()
        mixer.music.load(output_voice_file_changed)
       # mixer.music.load(output_voice_file) 
        mixer.music.play()
        while(mixer.music.get_busy()):
            pass
        mixer.music.unload()
        print("audio end")
        #playsound(output_voice_file)
        #playsound("response.wav")
        
        

def voice_prep(response):
    language = 'en'

    speech = gTTS(text=response, lang=language)

    speech.save(output_voice_file)
    #save_shifted_pitch("response", 7, )
    #os.system("start response_shifted.wav")
    #os.system("start " + output_voice_file)
    #playsound(output_voice_file)




def printText(frame, text, pos = 50):
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (pos, 1000)
    fontScale = 2
    color = (0, 255, 255)
    thickness = 4
    frame = cv2.putText(frame, text, org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
    return frame
    

    
class States(Enum):
    IDLE = 1
    LISTENING = 2
    SPEAKING = 3

def unicornVideo():
    cap = cv2.VideoCapture(idle_video_file)

    cv2.namedWindow("Unicorn", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Unicorn", 600, 600)
    #listen_seq=False
    #speaking_seq = False
    speaking_thread = None
    listening_thread = None
    state = States.IDLE
    # main loop
    text_response = None
    speed=12
    pos = 50
    while(cap.isOpened()):
        if state == States.LISTENING:
            if not listening_thread.is_alive():
                #print("thread not alive")
                text_response = listening_thread.response
                listening_thread = None
                state = States.SPEAKING
                speaking_thread = PlaySoundThread()
                speaking_thread.start()
                cap.release()
                cap = cv2.VideoCapture(speaking_video_file)
        elif state == States.SPEAKING:
            if not speaking_thread.is_alive():
                speaking_thread = None
                state = States.IDLE
                cap.release()
                cap = cv2.VideoCapture(idle_video_file)
                
        success, frame = cap.read()
        if success:
            if state == States.IDLE:
                frame = printText(frame, "Press <Space> to speak")
            if state == States.LISTENING:
                frame = printText(frame, "listening")
                pos = 50
            if state == States.SPEAKING:
                if text_response:
                    pos -= speed
                    frame = printText(frame, text_response, pos)
                else:
                    frame = printText(frame, "speaking")
                
            cv2.imshow('Unicorn', frame)
            
            quitButton = cv2.waitKey(25) & 0xFF == ord('q')
            closeButton = cv2.getWindowProperty('Unicorn', cv2.WND_PROP_VISIBLE) < 1
            if quitButton or closeButton: 
                break
                
            spaceButton = cv2.waitKey(25) & 0xFF == ord(' ')
            if spaceButton and state == States.IDLE:
                state = States.LISTENING
                print("space")
                cap.release()
                cap = cv2.VideoCapture(listen_video_file)
                
                listening_thread = VoiceRecordingThread()
                listening_thread.start()
                print("thread is started")
                
                #success, frame = cap.read()
                #if success:
                 #   cv2.imshow('Unicorn', frame)
                  #  quitButton = cv2.waitKey(25) & 0xFF == ord('q')
                    
                   # cap.release()
                    #cap = cv2.VideoCapture(speaking_video_file)
                    #speaking_seq = True
                    #todo: when to stop?
        else:
            # loop
            cap = cv2.VideoCapture(idle_video_file)
            speaking_thread = None
            listening_thread = None
            state = States.IDLE

    cap.release()
    cv2.destroyAllWindows()


def shift_pitch(filename, higher_pitch_shift, lower_pitch_shift, load_original=False):
    audio_wavelength, sampling_rate = librosa.load(filename)
    if load_original:
        IPython.display.display(Audio(data=audio_wavelength, rate=sampling_rate, autoplay=False))
    if higher_pitch_shift > 0:
        higher_pitch = librosa.effects.pitch_shift(audio_wavelength, sr=sampling_rate, n_steps=higher_pitch_shift)
        IPython.display.display(Audio(data=higher_pitch, rate=sampling_rate, autoplay=False))
    if lower_pitch_shift < 0:
        lower_pitch = librosa.effects.pitch_shift(audio_wavelength, sr=sampling_rate, n_steps=lower_pitch_shift)
        IPython.display.display(Audio(data=lower_pitch, rate=sampling_rate, autoplay=False))
        
def save_shifted_pitch(output_voice_file, output_voice_file_changed, pitch_shift, load_original=False, load_altered=False):
    audio_wavelength, sampling_rate = librosa.load(output_voice_file)
    altered = librosa.effects.pitch_shift(audio_wavelength, sr=sampling_rate, n_steps=pitch_shift)
    sf.write(output_voice_file_changed, altered, sampling_rate)

pygame 2.5.1 (SDL 2.28.2, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\Vadim\anaconda3\envs\assistant\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Vadim\anaconda3\envs\assistant\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Users\Vadim\anaconda3\envs\assistant\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [12]:
if __name__ == '__main__':
    unicornVideo()

In [ ]:
with sr.Microphone(device_index=0) as source:
    r.adjust_for_ambient_noise(source)
    print("speak")
    audio = r.listen(source)
user_input = r.recognize_google(audio)
print("User input:", user_input)
response = send_user_input('sunny-lightning-392907', 'us-central1', user_input)
print("Model response:", response)
voice(response)
playVideo()

In [19]:
import os
os.system("start response.wav")

0

In [23]:
shift_pitch('response.wav',
            higher_pitch_shift=7, #+6
            lower_pitch_shift=0)

In [ ]:
playsound("response.wav")

In [ ]:
!pip install gTTS


In [ ]:
from gtts import gTTS
language = 'en'

speech = gTTS(text=response, lang=language)

speech.save(output_voice_file)